Initialize client and load model.

In [3]:
from nos.client import Client

client = Client("[::]:50051")
model_id = "dream-gaussian"
model = client.Module(model_id)

Read example image and run inference.

In [4]:
import cv2

img = cv2.imread("./csm_luigi_rgba.png", cv2.IMREAD_UNCHANGED) # Input needs to be RGBA (4 channels) with transparency mask

response = model(img=img)

Unpack received response and save it to mesh.

In [9]:
import pygltflib
import numpy as np

f_np = response['f']
v_np = response['v']
vt_np = response['vt']
albedo = response['albedo']

albedo = (albedo * 255).astype(np.uint8)
albedo = cv2.cvtColor(albedo, cv2.COLOR_RGB2BGR)

f_np_blob = f_np.flatten().tobytes()
v_np_blob = v_np.tobytes()
# vn_np_blob = vn_np.tobytes()
vt_np_blob = vt_np.tobytes()
albedo_blob = cv2.imencode('.png', albedo)[1].tobytes()

gltf = pygltflib.GLTF2(
    scene=0,
    scenes=[pygltflib.Scene(nodes=[0])],
    nodes=[pygltflib.Node(mesh=0)],
    meshes=[pygltflib.Mesh(primitives=[
        pygltflib.Primitive(
            # indices to accessors (0 is triangles)
            attributes=pygltflib.Attributes(
                POSITION=1, TEXCOORD_0=2, 
            ),
            indices=0, material=0,
        )
    ])],
    materials=[
        pygltflib.Material(
            pbrMetallicRoughness=pygltflib.PbrMetallicRoughness(
                baseColorTexture=pygltflib.TextureInfo(index=0, texCoord=0),
                metallicFactor=0.0,
                roughnessFactor=1.0,
            ),
            alphaCutoff=0,
            doubleSided=True,
        )
    ],
    textures=[
        pygltflib.Texture(sampler=0, source=0),
    ],
    samplers=[
        pygltflib.Sampler(magFilter=pygltflib.LINEAR, minFilter=pygltflib.LINEAR_MIPMAP_LINEAR, wrapS=pygltflib.REPEAT, wrapT=pygltflib.REPEAT),
    ],
    images=[
        # use embedded (buffer) image
        pygltflib.Image(bufferView=3, mimeType="image/png"),
    ],
    buffers=[
        pygltflib.Buffer(byteLength=len(f_np_blob) + len(v_np_blob) + len(vt_np_blob) + len(albedo_blob))
    ],
    # buffer view (based on dtype)
    bufferViews=[
        # triangles; as flatten (element) array
        pygltflib.BufferView(
            buffer=0,
            byteLength=len(f_np_blob),
            target=pygltflib.ELEMENT_ARRAY_BUFFER, # GL_ELEMENT_ARRAY_BUFFER (34963)
        ),
        # positions; as vec3 array
        pygltflib.BufferView(
            buffer=0,
            byteOffset=len(f_np_blob),
            byteLength=len(v_np_blob),
            byteStride=12, # vec3
            target=pygltflib.ARRAY_BUFFER, # GL_ARRAY_BUFFER (34962)
        ),
        # texcoords; as vec2 array
        pygltflib.BufferView(
            buffer=0,
            byteOffset=len(f_np_blob) + len(v_np_blob),
            byteLength=len(vt_np_blob),
            byteStride=8, # vec2
            target=pygltflib.ARRAY_BUFFER,
        ),
        # texture; as none target
        pygltflib.BufferView(
            buffer=0,
            byteOffset=len(f_np_blob) + len(v_np_blob) + len(vt_np_blob),
            byteLength=len(albedo_blob),
        ),
    ],
    accessors=[
        # 0 = triangles
        pygltflib.Accessor(
            bufferView=0,
            componentType=pygltflib.UNSIGNED_INT, # GL_UNSIGNED_INT (5125)
            count=f_np.size,
            type=pygltflib.SCALAR,
            max=[int(f_np.max())],
            min=[int(f_np.min())],
        ),
        # 1 = positions
        pygltflib.Accessor(
            bufferView=1,
            componentType=pygltflib.FLOAT, # GL_FLOAT (5126)
            count=len(v_np),
            type=pygltflib.VEC3,
            max=v_np.max(axis=0).tolist(),
            min=v_np.min(axis=0).tolist(),
        ),
        # 2 = texcoords
        pygltflib.Accessor(
            bufferView=2,
            componentType=pygltflib.FLOAT,
            count=len(vt_np),
            type=pygltflib.VEC2,
            max=vt_np.max(axis=0).tolist(),
            min=vt_np.min(axis=0).tolist(),
        ),
    ],
)

# set actual data
gltf.set_binary_blob(f_np_blob + v_np_blob + vt_np_blob + albedo_blob)

gltf.save("./example.glb")

True

![output example](./example.gif)